### Runtime test to check wether recreating the board state based on uci or
### copying and pushing is faster in python-chess

In [ ]:
import chess
import chess.variant
import matplotlib.pyplot as plt
import random
import numpy as nps
from copy import deepcopy

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
def game_stats_random_play(board_in, nb_games, use_mate_shortcut=True):
    half_moves_stats = np.zeros(nb_games)
    game_outcomes = []
    final_board_states = []
    nb_legal_moves = []
    
    for i in range(nb_games):
        
        # create deepcopy of the board
        board = deepcopy(board_in)
        # reset the half moves counter
        nb_half_moves = 0
        while board.is_checkmate() is False and board.can_claim_draw() is False:

            str_moves = str(board.legal_moves)

            # select a random legal move
            mv_list = list(board.legal_moves)

            # log the number of legal moves
            nb_legal_moves.append(len(mv_list))
            
            mate_mv_idx = str_moves.find('#')
            
            if  len(mv_list) > 0:
                # check if there's a possible mate on the board -> execute it
                if mate_mv_idx != -1 and use_mate_shortcut is True:
                    # find the according index of the move in the legal_moves generator list
                    mv_idx = str_moves[:mate_mv_idx].count(',')
                else:
                    # take a random legal move
                    mv_idx = np.random.randint(len(mv_list))

                # apply the move to the board
                board.push(mv_list[mv_idx])
                nb_half_moves += 1
            else:
                break

        # record the number of half moves to the list
        half_moves_stats[i] = nb_half_moves
        # log the winner of the last board state
        game_outcomes.append(board.result())
        # log the final board state to the list
        final_board_states.append(board)
        
    return half_moves_stats, game_outcomes, final_board_states, nb_legal_moves

In [ ]:
def analyze_varariants(game_variant: dict, use_mate_shortcut=True, nb_sims=10):
    for game_name in game_variant:
        game = game_variant[game_name]
        game['half_moves_stats'], game['game_outcomes'], game['final_board_states'], game['nb_legal_moves'] = game_stats_random_play(game['board'], nb_sims, use_mate_shortcut)
        
        
    return game_variant

In [ ]:
game_variant = {}
#game_variant['antichess'] = {'board': chess.variant.GiveawayBoard()}
#game_variant['king of the hill'] = {'board': chess.variant.KingOfTheHillBoard()}
game_variant['chess'] = {'board': chess.Board()}
game_variant['atomic'] = {'board': chess.variant.AtomicBoard()}
game_variant['crazyhouse'] = {'board': chess.variant.CrazyhouseBoard()}
game_variant = analyze_varariants(game_variant, use_mate_shortcut=True, nb_sims=30)

In [ ]:
# select all  the stats
stats = list(game_variant[list(game_variant.keys())[0]].keys())

for stat in stats:
    
    if 'board' not in stat:
        # show a new histogram plot
        plt.figure()
        for variant in game_variant:
            game_variant[variant][stat] = plt.hist(game_variant[variant][stat], alpha=1/len(game_variant), label=variant)
            #xmax_list.append(max(game_variant[variant]['nb_legal_moves_stats'][1]))
            #ymax_list.append(max(game_variant[variant]['nb_legal_moves_stats'][0]))
            plt.legend()
            plt.title(stat)

## Comparision speed test copy vs recreation

In [ ]:
from time import time
import datetime

In [ ]:
board_in = chess.variant.CrazyhouseBoard()

t_s = time()

nb_half_moves = 0
boards = []
nb_sims = 4000
nb_moves_per_game = 10
moves = []
for i in range(nb_sims):
    # create deepcopy of the board
    board = deepcopy(board_in)

    for z in range(nb_moves_per_game):

        # select a random legal move
        mv_list = list(board.legal_moves)

        if  len(mv_list) > 0:
            # take a random legal move
            mv_idx = np.random.randint(len(mv_list))

            move = mv_list[mv_idx]
            # apply the move to the board
            board.push(move)
            nb_half_moves += 1
            
            boards.append(board.fen())
            moves.append(move)
        else:
            break


print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))            

In [ ]:
mv_idx = 0

t_s = time()

# create deepcopy of the board
board = deepcopy(board_in)
for board_fen in boards:
    board.set_fen(board_fen)
    mv_idx += 1

print('Elapsed time (hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))                    

In [ ]:
nb_sims

In [ ]:
t_s = time()

for i in range(nb_sims*10):
    # create deepcopy of the board
    board = deepcopy(board_in)    
    board.set_fen(board_fen)
print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))                    

In [ ]:
chess.variant.CrazyhouseBoard.

In [ ]:
board = chess.variant.CrazyhouseBoard()

In [ ]:
board.push_uci('e2e4')

In [ ]:
board

In [ ]:
mv = chess.Move.from_uci('c3c5')

In [ ]:
board.push(mv)

In [ ]:
t_s = time()
for i in range(nb_sims*100):
    board = deepcopy(board_in)
    board.push(mv)
print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))                        

In [ ]:
t_s = time()
for i in range(nb_sims*100):
    board = deepcopy(board_in)
    board.push_uci('e2e4')
print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))      

In [ ]:
board.push_uci('c3c5')

In [ ]:
t_s = time()

for i in range(nb_sims*10):
    # create deepcopy of the board
    board = deepcopy(board_in)
    for z in range(nb_moves_per_game):
        board.push(moves[z])
print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))    

In [ ]:
mv_idx = 0

t_s = time()

for i in range(nb_sims):
    # create deepcopy of the board
    board = deepcopy(board_in)
    for z in range(nb_moves_per_game):
        try:
            board.push(moves[mv_idx])
            mv_idx += 1
        except:
            pass
        
print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s)))) 

In [ ]:
len(moves)

In [ ]:
mv_idx

In [ ]:
board_in = chess.variant.CrazyhouseBoard()

t_s = time()

nb_half_moves = 0
boards = []
nb_sims = 1000
nb_moves_per_game = 42

for board_fen in boards:
    # create deepcopy of the board
    board = deepcopy(board_in)

    board.set_fen(board.fen())
    
    for z in range(nb_moves_per_game):

        # select a random legal move
        mv_list = list(board.legal_moves)

        if  len(mv_list) > 0:
            # take a random legal move
            mv_idx = np.random.randint(len(mv_list))

            # apply the move to the board
            board.push(mv_list[mv_idx])
            nb_half_moves += 1
        else:
            break

        boards.append(board)

print('Elapsed time(hh:mm:ss): ' + str(datetime.timedelta(seconds=round(time() - t_s))))    